### Import libraries

In [ ]:
from ipywidgets import interactive
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import scipy.signal as sps
from IPython.display import HTML

### Parameters

In [ ]:
time_limit = 4
frequency_range = 25 

uXLim = [-time_limit, time_limit]
lXLim=[-frequency_range, frequency_range]
period = 1

time_axis = np.round(np.arange(-0.5, 0.5+0.005, 0.005), decimals=3)

delta = ((time_axis[-1]-time_axis[0]) / 
         (len(time_axis)-1))

waveform_options = ['Rectangular pulse', 'Triangle', 'Raised cosine']

waveform = waveform_options[1]

# animation iteraton interval between 0,1
step = 0.005

### Function Definition

In [ ]:
def td_waveform(Waveform):
    wf = Waveform
    s = np.zeros(time_axis.size)
    
    if wf == waveform_options[0]:
        # Define a rectangular pulse in time
        s[abs(time_axis)<=0.1] = 1
        
    elif wf == waveform_options[1]:
        # Define a triangular pulse in time
        s[abs(time_axis)<=0.2] = 1 - 5 * abs(time_axis[abs(time_axis)<=0.2])
        
    elif wf ==  waveform_options[2]:
        # Define a raised cosine pulse in time
        s[abs(time_axis)<=0.2] = 1 + np.cos(np.pi*time_axis[abs(time_axis)<=0.2]/0.2)
        
    return s

In [ ]:
def Perform_animation(Waveform):
    time_signal = td_waveform(Waveform)
    
    # Generate a closely spaced frequency response
    zero_padded = np.zeros(4096)
    zero_padded[0:int((len(time_signal)+1)/2)] = time_signal[int((len(time_signal)+1)/2-1):]
    zero_padded[int(-1-((len(time_signal)-3)/2)):] = time_signal[0:int((len(time_signal)-1)/2)]

    # We assume a symmetric signal
    frequency_response = (np.fft.fftshift(np.fft.fft(zero_padded))).real
    fmin = min(frequency_response)
    fmax = max(frequency_response)

    if (fmin < 0):
        fmin = fmin * 1.05
    else:
        fmin = fmin * 0.95

    if (fmax < 0):
        fmax = fmax * 0.95
    else:
        fmax = fmax * 1.05

    flimits = [fmin, fmax]
    frequency_axis = (np.arange(-(len(zero_padded)/2),(len(zero_padded)/2))
                     / (delta*len(zero_padded)))


    fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(16, 8))
    TimeAX, = ax1.plot([], [], 'b')
    ax1.set_title('Time Domain')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    ax1.set_xlim(uXLim)
    
    if Waveform == 'Raised cosine':
        ax1.set_ylim(0, 2)
    else: 
        ax1.set_ylim(0, 1.01)

    (markerLines, stemLines, baseLines) = ax2.stem([0], [0])
    plt.setp(stemLines, color = 'blue', linewidth=1.3) 
    plt.setp(baseLines, color = 'black', linewidth=1.3)
    plt.setp(markerLines, color = 'blue') 
    markerLines.set_markerfacecolor('none')

    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Amplitude')
    ax2.set_title('Frequency Domain')
    ax2.set_xlim(lXLim)
    
    plt.tight_layout()
    def animate(i):
        value = (i) * step
        period = np.round(1 + time_limit*value, 4)

        # The maximum number of repetitions is given by 2*time_limit+1,
        # so generate this many copies, with the apropriate number of
        # zeros between repetitions
        reps = 2*time_limit + 1
        separation = np.round((period-1)/delta, 0)
        time_sep = len(time_signal) + separation
        time_data = np.zeros(int(len(time_signal) * reps + separation * (reps-1)))
        ptr = 1
        for idx in range(reps):
            time_data[int(ptr-1) : int(ptr+len(time_signal)-1)] = time_signal
            ptr = ptr + time_sep

        time_scale = ((np.arange(0, (len(time_data))) - ((len(time_data)-1)/2)) 
                      * delta)

        # Time Domain plot
        TimeAX.set_data(time_scale, time_data)

        # Frequency domain stem plot
        f_sample_spacing = len(frequency_response) / (len(time_signal)+separation)
        upper = np.arange(0, (frequency_range * delta * 
                              len(frequency_response)), f_sample_spacing)
        upper = upper.reshape(1, len(upper))

        faxis = np.concatenate((-np.fliplr(upper), upper[0, 1:].reshape(1, upper.size-1)), axis = 1)

        findex = np.round(faxis) + len(frequency_response)/2
        faxis = faxis / (delta*len(frequency_response))
        fvalues = frequency_response[findex.astype(int)] / time_sep

        lYLim = flimits / time_sep
        ax2.set_ylim(lYLim) 

        ## update stem plot
        markerLines.set_data(faxis[0], fvalues[0])
        baseLines.set_data([-len(fvalues[0]), len(fvalues[0])], [0,0])
        stem_len = len(stemLines)

        for i in range(len(fvalues[0])):
            if i < stem_len:
                stemLines[i].set_data([faxis[0][i], faxis[0][i]], [0, fvalues[0][i]])
            else:
                stemLines.append(matplotlib.lines.Line2D([faxis[0][i], faxis[0][i]],
                                                         [0, fvalues[0][i]], color = "blue"))
                ax2.add_line(stemLines[i])

        plt.tight_layout()

    ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                             frames = int(1/step)+1)
    plt.close()

    display(HTML(ani.to_jshtml()))



In [ ]:
### if cell show result in scrolling mode, click menu cell - Current Outputs - Toggle twice
matplotlib.rcParams.update({'font.size': 15})
interactive(Perform_animation, Waveform=waveform_options)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.